<a href="https://colab.research.google.com/github/alexmacharia/deep_learning_projects/blob/main/9_CNN_Visualisation_Filter_and_Filter_Activation_Visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

print("Initial shape or dimensions of x_train", str(x_train.shape))

print("Number of samples in training data: " + str(len(x_train)))

11490434/11490434 [==============================] - 0s 0us/step
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14530577059232444515
xla_global_id: -1
]
Initial shape or dimensions of x_train (60000, 28, 28)
Number of samples in training data: 60000


In [2]:
# Store number of rows and columns
img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]

# Reshape data for Keras
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

# Store shape of single image
input_shape = (img_rows, img_cols, 1)

# change image type to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the data
x_train /= 255.0
x_test /= 255.0

print('x_train.shape', x_train.shape)
print(x_train.shape[0], 'training samples')
print(x_test.shape[0], 'test samples')


x_train.shape (60000, 28, 28, 1)
60000 training samples
10000 test samples


In [3]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print("Number of Classes: ", str(y_test.shape[1]))

num_classes = y_test.shape[1]
num_pixels = x_train.shape[1] * x_train.shape[2]

Number of Classes:  10


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(0.001),
              metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                        

In [5]:
batch_size = 128
epochs = 10

history = model.fit(x_train,
                    y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
469/469 [==============================] - 101s 214ms/step - loss: 2.2412 - accuracy: 0.3383 - val_loss: 2.1410 - val_accuracy: 0.5917
Epoch 2/10
469/469 [==============================] - 93s 199ms/step - loss: 1.8361 - accuracy: 0.6712 - val_loss: 1.2878 - val_accuracy: 0.7781
Epoch 3/10
469/469 [==============================] - 92s 196ms/step - loss: 0.8430 - accuracy: 0.8145 - val_loss: 0.5503 - val_accuracy: 0.8628
Epoch 4/10
469/469 [==============================] - 92s 196ms/step - loss: 0.4952 - accuracy: 0.8655 - val_loss: 0.4119 - val_accuracy: 0.8883
Epoch 5/10
469/469 [==============================] - 90s 193ms/step - loss: 0.4081 - accuracy: 0.8843 - val_loss: 0.3595 - val_accuracy: 0.9002
Epoch 6/10
469/469 [==============================] - 92s 196ms/step - loss: 0.3675 - accuracy: 0.8939 - val_loss: 0.3308 - val_accuracy: 0.9061
Epoch 7/10
469/469 [==============================] - 91s 194ms/step - loss: 0.3423 - accuracy: 0.9005 - val_loss: 0.3121 - val_a

In [6]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

{'conv2d': <keras.src.layers.convolutional.conv2d.Conv2D at 0x7c78af229f00>,
 'conv2d_1': <keras.src.layers.convolutional.conv2d.Conv2D at 0x7c782907da50>,
 'max_pooling2d': <keras.src.layers.pooling.max_pooling2d.MaxPooling2D at 0x7c782907de40>,
 'flatten': <keras.src.layers.reshaping.flatten.Flatten at 0x7c782907f100>,
 'dense': <keras.src.layers.core.dense.Dense at 0x7c782907fc70>,
 'dense_1': <keras.src.layers.core.dense.Dense at 0x7c782907fd30>}

In [7]:
for layer in model.layers:

  if 'conv' not in layer.name:
    continue

  filters, biases = layer.get_weights()
  print(layer.name, filters.shape)

conv2d (3, 3, 1, 32)
conv2d_1 (3, 3, 32, 64)
